# Regression, Fit
    




In [2]:


import pandas as pd
import numpy as np
import os
from openpyxl import load_workbook
import openpyxl
import time
from datetime import datetime



###################### OSR ###########################

def multi_lot_regression(df_rawdata):
    # UNIQUE_ID별로 그룹화
    grouped = df_rawdata.groupby('UNIQUE_ID')

    # 회귀분석 결과를 저장할 리스트
    wkrk_results = []

    # 각 그룹에 대해 처리
    for unique_id, group in grouped:
        die_x = group['DieX']
        die_y = group['DieY']
        step_pitch_x = group['STEP_PITCH_X']
        step_pitch_y = group['STEP_PITCH_Y']
        map_shift_x = group['MAP_SHIFT_X']
        map_shift_y = group['MAP_SHIFT_Y']
        field_x = group['coordinate_X']
        field_y = group['coordinate_Y']
        
        # 좌표 계산
        x = die_x * step_pitch_x + map_shift_x
        y = die_y * step_pitch_y + map_shift_y
        rx = field_x
        ry = field_y

        # X_dx, X_dy 행렬 구성
        X_dx = np.vstack([
            np.ones(len(x)), x/1e6, -y/1e6, (x**2)/1e12, (x*y)/1e12, (y**2)/1e12, (x**3)/1e15, (x**2*y)/1e15, (x*y**2)/1e15, (y**3)/1e15, 
            rx/1e6, -ry/1e6, (rx**2)/1e9, (rx*ry)/1e9, (ry**2)/1e9, (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12, (ry**3)/1e12
        ]).T
        X_dy = np.vstack([
            np.ones(len(y)), y/1e6, x/1e6, (y**2)/1e12, (y*x)/1e12, (x**2)/1e12, (y**3)/1e15, (y**2*x)/1e15, (y*x**2)/1e15, (x**3)/1e15,
            ry/1e6, rx/1e6, (ry**2)/1e9, (ry*rx)/1e9, (rx**2)/1e9, (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12
        ]).T

        # 종속변수
        Y_dx = group['X_reg']
        Y_dy = group['Y_reg']

        # 최소자승법으로 계수 계산
        coeff_dx = np.linalg.lstsq(X_dx, Y_dx, rcond=None)[0]
        coeff_dy = np.linalg.lstsq(X_dy, Y_dy, rcond=None)[0]

        # 결과 저장
        wkrk_results.append(pd.DataFrame({
            'UNIQUE_ID': [unique_id],
            'WK1': [coeff_dx[0]],
            'WK2': [coeff_dy[0]],
            'WK3': [coeff_dx[1]],
            'WK4': [coeff_dy[1]],
            'WK5': [coeff_dx[2]],
            'WK6': [coeff_dy[2]],
            'WK7': [coeff_dx[3]],
            'WK8': [coeff_dy[3]],
            'WK9': [coeff_dx[4]],
            'WK10': [coeff_dy[4]],
            'WK11': [coeff_dx[5]],
            'WK12': [coeff_dy[5]],
            'WK13': [coeff_dx[6]],
            'WK14': [coeff_dy[6]],
            'WK15': [coeff_dx[7]],
            'WK16': [coeff_dy[7]],
            'WK17': [coeff_dx[8]],
            'WK18': [coeff_dy[8]],
            'WK19': [coeff_dx[9]],
            'WK20': [coeff_dy[9]],
            'RK1': [0],
            'RK2': [0],
            'RK3': [coeff_dx[10]],
            'RK4': [coeff_dy[10]],
            'RK5': [coeff_dx[11]],
            'RK6': [coeff_dy[11]],
            'RK7': [coeff_dx[12]],
            'RK8': [coeff_dy[12]],
            'RK9': [coeff_dx[13]],
            'RK10': [coeff_dy[13]],
            'RK11': [coeff_dx[14]],
            'RK12': [coeff_dy[14]],
            'RK13': [coeff_dx[15]],
            'RK14': [coeff_dy[15]],
            'RK15': [coeff_dx[16]],
            'RK16': [coeff_dy[16]],
            'RK17': [coeff_dx[17]],
            'RK18': [coeff_dy[17]],
            'RK19': [coeff_dx[18]],
            'RK20': [0],
        }))

    # 결과 병합
    combined_results = pd.concat(wkrk_results, ignore_index=True)
    return combined_results



###################### OSR Fitting, Residual ###########################


def multi_lot_fitting_residual(df_rawdata, df_coeff):
    # UNIQUE_ID별로 그룹화
    grouped = df_rawdata.groupby('UNIQUE_ID')
    
    # 예측 결과를 저장할 리스트
    predictions_list = []
    
    for unique_id, group in grouped:
 

        die_x = group['DieX']
        die_y = group['DieY']
        step_pitch_x = group['STEP_PITCH_X']
        step_pitch_y = group['STEP_PITCH_Y']
        map_shift_x = group['MAP_SHIFT_X']
        map_shift_y = group['MAP_SHIFT_Y']
        coordinate_x = group['coordinate_X']
        coordiante_y = group['coordinate_Y']   
              
        
        x = die_x * step_pitch_x + map_shift_x
        y = die_y * step_pitch_y + map_shift_y
        rx = coordinate_x
        ry = coordiante_y

        X_dx = np.vstack([
            np.ones(len(x)), x/1e6, -y/1e6, (x**2)/1e12, (x*y)/1e12, (y**2)/1e12, (x**3)/1e15, (x**2*y)/1e15, (x*y**2)/1e15, (y**3)/1e15, 
            rx/1e6, -ry/1e6, (rx**2)/1e9, (rx*ry)/1e9, (ry**2)/1e9, (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12, (ry**3)/1e12
        ]).T
        X_dy = np.vstack([
            np.ones(len(y)), y/1e6, x/1e6, (y**2)/1e12, (y*x)/1e12, (x**2)/1e12, (y**3)/1e15, (y**2*x)/1e15, (y*x**2)/1e15, (x**3)/1e15,
            ry/1e6, rx/1e6, (ry**2)/1e9, (ry*rx)/1e9, (rx**2)/1e9, (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12
        ]).T

        # 해당 LOT_ID의 계수 추출
        coeffs = df_coeff[df_coeff['UNIQUE_ID'] == unique_id].iloc[0]
        coeff_dx = coeffs[['WK1','WK3','WK5','WK7','WK9','WK11','WK13','WK15','WK17','WK19','RK3','RK5','RK7','RK9','RK11','RK13','RK15','RK17','RK19']].values.astype(float)
        coeff_dy = coeffs[['WK2','WK4','WK6','WK8','WK10','WK12','WK14','WK16','WK18','WK20','RK4','RK6','RK8','RK10','RK12','RK14','RK16','RK18']].values.astype(float)


        # 예측값 계산
        pred_x = X_dx.dot(coeff_dx)
        pred_y = X_dy.dot(coeff_dy)

        # 잔차 계산
        residual_x = group['X_reg'] - pred_x
        residual_y = group['Y_reg'] - pred_y

        # 결과 저장
        predictions_list.append(pd.DataFrame({
            'pred_x': pred_x,
            'pred_y': pred_y,
            'residual_x': residual_x,
            'residual_y': residual_y,
        }))

    # 예측 결과 병합
    df_predictions = pd.concat(predictions_list, ignore_index=True)
    return df_predictions








################################### MULTI LOT REGRESSION #####################################################################

# 데이터 불러오기 (엑셀 -> CSV)
df_rawdata = pd.read_csv("RawData-1_2lot.csv")


# 회귀분석 결과 저장 (엑셀 -> CSV)
df_coeff = multi_lot_regression(df_rawdata)
df_coeff.to_csv('OSR_K.csv', index=False)


################################### MULTI LOT FITTING, RESIDUAL #####################################################################

# 잔차 계산
df_predictions = multi_lot_fitting_residual(df_rawdata, df_coeff)
df_rawdata = pd.concat([df_rawdata, df_predictions], axis=1)  # axis=1은 열 단위로 병합하는 것을 의미 (행 단위 병합은 axis=0)
df_rawdata.to_csv('OSR_FIT.csv', index=False)    



